In [33]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
import joblib

In [2]:
import kagglehub

path = kagglehub.dataset_download("olistbr/brazilian-ecommerce")
print("Path to dataset files:", path)

customers = pd.read_csv(f"{path}/olist_customers_dataset.csv")
orders = pd.read_csv(f"{path}/olist_orders_dataset.csv")
order_items = pd.read_csv(f"{path}/olist_order_items_dataset.csv")
payments = pd.read_csv(f"{path}/olist_order_payments_dataset.csv")
reviews = pd.read_csv(f"{path}/olist_order_reviews_dataset.csv")
sellers = pd.read_csv(f"{path}/olist_sellers_dataset.csv")

C:\Users\Qamar Hasan\AppData\Roaming\Python\Python313\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100%|██████████| 42.6M/42.6M [00:27<00:00, 1.62MB/s]

Extracting files...


Path to dataset files: C:\Users\Qamar Hasan\.cache\kagglehub\datasets\olistbr\brazilian-ecommerce\versions\2


In [3]:
customers.head()

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP


In [4]:
orders.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00


In [5]:
order_items.head()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14


In [6]:
payments.head()

,order_id,payment_sequential,payment_type,payment_installments,payment_value
0,b81ef226f3fe1789b1e8b2acac839d17,1,credit_card,8,99.33
1,a9810da82917af2d9aefd1278f1dcfa0,1,credit_card,1,24.39
2,25e8ea4e93396b6fa0d3dd708e76c1bd,1,credit_card,1,65.71
3,ba78997921bbcdc1373bb41e913ab953,1,credit_card,8,107.78
4,42fdf880ba16b47b59251dd489d4441a,1,credit_card,2,128.45


In [8]:
reviews.head()

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,NaN,NaN,2018-01-18 00:00:00,2018-01-18 21:46:59
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,NaN,NaN,2018-03-10 00:00:00,2018-03-11 03:05:13
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,NaN,NaN,2018-02-17 00:00:00,2018-02-18 14:36:24
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,NaN,Recebi bem antes do prazo estipulado.,2017-04-21 00:00:00,2017-04-21 22:02:06
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,NaN,Parabéns lojas lannister adorei comprar pela I...,2018-03-01 00:00:00,2018-03-02 10:26:53


In [9]:
sellers.head()

,seller_id,seller_zip_code_prefix,seller_city,seller_state
0,3442f8959a84dea7ee197c632cb2df15,13023,campinas,SP
1,d1b65fc7debc3361ea86b5f14c68d2e2,13844,mogi guacu,SP
2,ce3ad9de960102d0677a81f5d0bb7b2d,20031,rio de janeiro,RJ
3,c0f3eea2e14555b6faeea3dd58c1b1c3,4195,sao paulo,SP
4,51a04a8a6bdcb23deccc82b0b80742cf,12914,braganca paulista,SP


We convert string dates to datetime objects so we can calculate durations, recency, and delays.

Allows recency calculation, shipping delays, and time-based features—all strong indicators of churn.

In [10]:
# Preprocess Dates
orders['order_purchase_timestamp'] = pd.to_datetime(orders['order_purchase_timestamp'])
orders['order_delivered_customer_date'] = pd.to_datetime(orders['order_delivered_customer_date'])
orders['order_estimated_delivery_date'] = pd.to_datetime(orders['order_estimated_delivery_date'])

snapshot_date = orders['order_purchase_timestamp'].max()


Recency: How long since last purchase.

Frequency: How often a customer buys.

Monetary: How much a customer spends.



RFM features are classic churn predictors. Inactive, low-frequency, or low-spending customers are more likely to churn.

In [11]:
# Basic RFM Features
# Recency
recency = orders.groupby('customer_id')['order_purchase_timestamp'].max().apply(lambda x: (snapshot_date - x).days)

# Frequency
frequency = orders.groupby('customer_id')['order_id'].nunique()

# Monetary
orders_items_merge = orders.merge(order_items[['order_id', 'price']], on='order_id', how='left')
monetary = orders_items_merge.groupby('customer_id')['price'].sum()

Cancel rate: Measures dissatisfaction or indecisiveness.

Avg review: Indicates customer satisfaction.

In [12]:
# Cancel Rate & Reviews
cancel_rate = orders.groupby('customer_id')['order_status'].apply(lambda x: (x=='canceled').sum()/x.count())
avg_review = orders.merge(reviews[['order_id','review_score']], on='order_id', how='left')
avg_review = avg_review.groupby('customer_id')['review_score'].mean()


Payment patterns help understand how customers prefer to pay and average spending per transaction.



Benefit:
Customers who abandon certain payment types (like Boleto) or spend inconsistently may be more likely to churn.

In [13]:
# Payment Behavior
payment_info = orders.merge(payments, on='order_id', how='left')
most_used_payment = payment_info.groupby('customer_id')['payment_type'].agg(lambda x: x.mode()[0] if not x.mode().empty else 'unknown')
avg_payment_value = payment_info.groupby('customer_id')['payment_value'].mean()


Why:
Location can affect customer behavior (delivery times, accessibility, preferences).

Benefit:
Allows geographic insights—helps capture regional churn patterns.

In [14]:
# Region / Zip

region = customers.set_index('customer_id')['customer_zip_code_prefix']

Why:
Combine all basic features into a single customer-level dataset.

Benefit:
Provides a baseline feature set for predicting churn before adding advanced features.

In [15]:
# Merge Simple Features
features = pd.DataFrame({
    'recency': recency,
    'frequency': frequency,
    'monetary': monetary,
    'cancel_rate': cancel_rate,
    'avg_review_score': avg_review,
    'most_used_payment': most_used_payment,
    'avg_payment_value': avg_payment_value,
    'region': region
})

Why:
Measures how long a customer has been active.

Benefit:
Longer-lifetime but inactive customers may be at higher risk of churn—adds nuance beyond recency alone.

In [16]:
# Advanced Features – Customer Lifetime
first_last = orders.groupby('customer_id').agg(first_order=('order_purchase_timestamp','min'),
                                              last_order=('order_purchase_timestamp','max'))
features['customer_lifetime_days'] = (first_last['last_order'] - first_last['first_order']).dt.days


Why:
Calculates ordering consistency and gaps.

Benefit:
Frequent or regular buyers are less likely to churn; irregular patterns may signal churn risk.

In [17]:
# Time Between Orders
orders_sorted = orders.sort_values(['customer_id','order_purchase_timestamp'])
orders_sorted['prev_order'] = orders_sorted.groupby('customer_id')['order_purchase_timestamp'].shift(1)
orders_sorted['days_between'] = (orders_sorted['order_purchase_timestamp'] - orders_sorted['prev_order']).dt.days
freq_features = orders_sorted.groupby('customer_id').agg(
    avg_time_between_orders=('days_between','mean'),
    median_time_between_orders=('days_between','median'),
    std_time_between_orders=('days_between','std')
)
features = features.merge(freq_features, left_index=True, right_index=True, how='left')

Why:
Captures short-term activity trends.

Benefit:
A drop in recent orders relative to past behavior is a strong churn indicator.

In [18]:
# Orders in Last X Days
for window in [30,60,90,120]:
    mask = orders['order_purchase_timestamp'] >= (snapshot_date - pd.Timedelta(days=window))
    counts = orders[mask].groupby('customer_id')['order_id'].nunique().rename(f'orders_last_{window}d')
    features = features.merge(counts, left_index=True, right_index=True, how='left')
features.fillna(0, inplace=True)

Why:
Measures delivery performance: delays, shipping times, and undelivered orders.

Benefit:
Poor delivery experiences increase churn likelihood.

In [19]:
# Shipping & Delivery Features
shipping_df = orders.merge(order_items[['order_id','price']], on='order_id', how='left')
shipping_df['shipping_days'] = (shipping_df['order_delivered_customer_date'] - shipping_df['order_purchase_timestamp']).dt.days
shipping_df['delivery_delay'] = (shipping_df['order_delivered_customer_date'] - shipping_df['order_estimated_delivery_date']).dt.days
logistics = shipping_df.groupby('customer_id').agg(
    avg_shipping_days=('shipping_days','mean'),
    median_shipping_days=('shipping_days','median'),
    std_shipping_days=('shipping_days','std'),
    avg_delivery_delay=('delivery_delay','mean'),
    late_delivery_ratio=('delivery_delay', lambda x: (x>0).mean()),
    undelivered_orders_ratio=('order_delivered_customer_date', lambda x: x.isna().mean())
)
features = features.merge(logistics, left_index=True, right_index=True, how='left')

Why:
Captures dependence on a single seller or variety.

Benefit:
Customers buying from only one seller may be more sensitive to bad experience, which can trigger churn.

In [20]:
# Seller Diversity
orders_sellers = orders.merge(order_items[['order_id','product_id', 'seller_id']], on='order_id', how='left').merge(sellers, on='seller_id', how='left')
seller_features = orders_sellers.groupby('customer_id').agg(
    unique_sellers=('seller_id','nunique'),
    unique_seller_states=('seller_state','nunique')
)
features = features.merge(seller_features, left_index=True, right_index=True, how='left')

Why:
Creates explainable churn labels using business rules.

Benefit:

Makes churn detection interpretable for stakeholders.

Combines multiple factors: inactivity, declining activity, monetary, satisfaction, delivery, seller dependence.

Produces a target variable for ML modeling.

In [21]:
dynamic_cutoff = features['avg_time_between_orders'].replace(0, 120) * 2

inactivity_flag    = features['recency'] > dynamic_cutoff*1.5
activity_decline   = (features['frequency'] > 3) & ((features['orders_last_90d']/features['frequency'])<0.3)
monetary_decline   = (features['monetary']>0) & (features['avg_payment_value'] < 0.3*features['monetary'])
bad_reviews        = (features['avg_review_score']<=3) & (features['recency']>dynamic_cutoff)
late_deliveries    = (features['late_delivery_ratio']>0.7) & (features['recency']>dynamic_cutoff)
undelivered_orders = features['undelivered_orders_ratio']>0.2
one_time_buyer     = (features['frequency']==1) & (features['recency']>365)
high_value_churn   = (features['avg_payment_value']>features['avg_payment_value'].median()) & inactivity_flag
seller_dependency  = (features['unique_sellers']==1) & (bad_reviews)

features['churn'] = (
    inactivity_flag | activity_decline | monetary_decline |
    bad_reviews | late_deliveries | undelivered_orders |
    one_time_buyer | high_value_churn | seller_dependency
).astype(int)


In [23]:
features.to_csv("customer_churn_optimized.csv")

In [46]:
# Load the processed dataset
df = pd.read_csv("D:/work/Github/Customer_segmentations/Churn/customer_churn_optimized.csv")


In [47]:
# ------------------------
# Align column names
# ------------------------
# Use the features we actually built
features = [
    "recency",             # same as your engineered recency
    "frequency",           # number of orders
    "monetary",            # total spent
    "avg_payment_value",   # average order value proxy
    "avg_review_score"
]

# ------------------------
# Prepare Features & Target
# ------------------------
X = df[features]
y = df["churn"]


In [48]:
# Handle missing values (basic way)
X = X.fillna(0)

# Scale numeric features (no pipeline, just fit/transform)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


In [49]:
# ------------------------
# Train/Test Split
# ------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, stratify=y, random_state=42
)


In [50]:
# ------------------------
# Define models
# ------------------------
models = {
    "Logistic Regression": LogisticRegression(max_iter=1000, class_weight='balanced'),
    "Random Forest": RandomForestClassifier(n_estimators=200, random_state=42, class_weight='balanced'),
    "XGBoost": XGBClassifier(
        n_estimators=200, 
        eval_metric='logloss', 
        random_state=42, 
        use_label_encoder=False
    )
}


In [51]:
# ------------------------
# Train & Evaluate
# ------------------------
results = []

for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    y_proba = model.predict_proba(X_test)[:, 1] if hasattr(model, "predict_proba") else None

    report = classification_report(y_test, y_pred, output_dict=True, zero_division=0)
    roc_auc = roc_auc_score(y_test, y_proba) if y_proba is not None else np.nan

    results.append({
        "Model": name,
        "Accuracy": report['accuracy'],
        "Precision": report['1']['precision'],
        "Recall": report['1']['recall'],
        "F1-score": report['1']['f1-score'],
        "ROC-AUC": roc_auc
    })

results_df = pd.DataFrame(results).sort_values(by='F1-score', ascending=False)
print("Model Comparison:")
print(results_df)

C:\Users\Qamar Hasan\AppData\Roaming\Python\Python313\site-packages\xgboost\training.py:183: UserWarning: [17:30:10] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Model Comparison:
                 Model  Accuracy  Precision    Recall  F1-score   ROC-AUC
1        Random Forest  0.985670   0.998285  0.965297  0.981514  0.990719
2              XGBoost  0.985117   0.998282  0.963894  0.980787  0.992112
0  Logistic Regression  0.936347   0.899951  0.943353  0.921141  0.982701


In [53]:
from sklearn.metrics import precision_recall_curve
import joblib

# Use your best model (say RandomForest)
best_model = RandomForestClassifier(
    n_estimators=200, random_state=42, class_weight='balanced'
)
best_model.fit(X_train, y_train)

# Get probabilities
y_proba = best_model.predict_proba(X_test)[:, 1]

# Find best threshold (F1-score based)
prec, rec, thresh = precision_recall_curve(y_test, y_proba)
f1_scores = 2 * (prec * rec) / (prec + rec + 1e-6)  # avoid div by 0
best_idx = np.argmax(f1_scores)
best_threshold = thresh[best_idx]

print("✅ Best threshold for churn:", best_threshold)


✅ Best threshold for churn: 0.67


In [52]:
# ------------------------
# Save the best model
# ------------------------
best_model = RandomForestClassifier(n_estimators=200, random_state=42, class_weight='balanced')
best_model.fit(X_scaled, y)  # train on all data
joblib.dump((best_model, scaler), "churn_model.pkl")   # save model + scaler
print("✅ Model and scaler saved as churn_model.pkl")

✅ Model and scaler saved as churn_model.pkl
